# AI PROJECT - CHURN PREDICTION

#### Purpose of the task in hand is to create a machine learning model that will be able to predict churn.
   Customer churn: a buyer who fails to place any new order within a chosen inactivity window (e.g., 90-180 days) after their last purchase date.


In [23]:
# %cd "/content/drive/MyDrive/Colab Notebooks/Final Project/Churn Prediction"
# !pwd
!git init
!echo -e ".ipynb_checkpoints/\nsample_data/\ndrive/\n.config/" > .gitignore
!git add .
!git commit -m "Initial commit: Churn Prediction project"


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Colab Notebooks/Final Project/Churn Prediction/.git/
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@78ee20dbaabe.(none)')


/content/drive/MyDrive/Colab Notebooks/Final Project/Churn Prediction
